In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
# Filepath of the dataset
pre_filepath = "../../../../../../Volumes/Seagate Backup Plus Drive/Documents/Kaggle Datasets/"

In [17]:
X_train = tf.placeholder(tf.float32, [1, 64, 64, 3])
y_train = tf.placeholder(tf.float32, [1, 17])

In [12]:
# Paramaters definitions
VALIDATION_SPLIT = 35000
BATCH_SIZE = 64
n_classes = int(y_train.shape[1])

In [44]:
def validation_data(images, split):
    """
    Arguments:
        images: Numpy array of all the input images.
        split: Number of input images to use for training. The rest will be used for validation.
    
    Returns:
        Tuple of 4 numpy arrays with format (X_train, X_valid, y_train, y_valid)
    """
    # Ensure validation split is valid
    num_examples = images.shape[0]
    if validation_split > num_examples:
        raise Exception('validation_split greater than number of examples')
    
    # Split the training sets into training and validation sets
    X_train, X_valid = X_train[:validation_split], X_train[validation_split:]
    y_train, y_valid = y_train[:validation_split], y_train[validation_split:]
    
    return (X_train, X_valid, y_train, y_valid)
    
# Helper wrappers
def conv2d(x, W, b, strides=1):
    return_val = tf.nn.conv2d(x, W, strides=[1,strides,strides,1], padding='SAME')
    return_val = tf.nn.bias_add(return_val, b)
    return tf.nn.relu(return_val)

def maxpool(x, pool_size=2):
    return tf.nn.max_pool(x, [1,pool_size,pool_size,1], [1,pool_size,pool_size,1], padding='SAME')

def fc(x, W, b):
    return_val = tf.matmul(x, W)
    return_val = tf.nn.bias_add(return_val, b)
    return tf.nn.relu(return_val)

In [45]:
# Build the Tensorflow model!
def model(images, weights, biases, dropout=0.8):
    """
    Defines the image classification model
    
    Inputs:
        images: entire training set of images
        input_shape: dimensions of input as a tuple
    
    Outputs logits
    """
    
    # Apply convolution and pooling to each layer
    conv1 = conv2d(images, weights['conv1'], biases['conv1'])  
    conv1 = maxpool(conv1)
    
    conv2 = conv2d(conv1, weights['conv2'], biases['conv2'])
    conv2 = maxpool(conv2)
    
    conv3 = conv2d(conv2, weights['conv3'], biases['conv3'])
    conv3 = maxpool(conv3)
    
    # Apply dropout
    conv3 = tf.nn.dropout(conv3, dropout)
    
    # First reshape output of conv3 into a vector
    conv3_vec = tf.reshape(conv3, [1, 8*8*128])
    
    # FC layers
    fc1 = fc(conv3_vec, weights['fc1'], biases['fc1'])
    # Then apply dropout
    fc1 = tf.nn.dropout(fc1, dropout)
    
    fc2 = fc(fc1, weights['fc2'], biases['fc2'])
    
    # Return logits which is a vector of 17 class scores
    return fc2

In [46]:
weights = {'conv1':tf.Variable(tf.random_normal([3,3,3,32])), # 3 by 3 convolution, 3 channels (depth), 32 outputs
           'conv2':tf.Variable(tf.random_normal([3,3,32,64])), # 3 by 3 convolution, 32 inputs, 64 outputs
           'conv3':tf.Variable(tf.random_normal([3,3,64,128])), # 3 by 3 convolution, 64 inputs, 128 outputs
           'fc1':tf.Variable(tf.random_normal([8*8*128,1024])), 
           'fc2':tf.Variable(tf.random_normal([1024,n_classes]))}

biases = {'conv1':tf.Variable(tf.random_normal([32])),
          'conv2':tf.Variable(tf.random_normal([64])),
          'conv3':tf.Variable(tf.random_normal([128])),
          'fc1':tf.Variable(tf.random_normal([1024])),
          'fc2':tf.Variable(tf.random_normal([n_classes]))}

In [47]:
# Instantiate the model
conv_model = model(X_train, weights, biases)

In [48]:
# Load X_train and y_train arrays from disk
# X_train = np.load(pre_filepath + "/Planet/model-data/X_train_arr.npy")
# y_train = np.load(pre_filepath + "/Planet/model-data/y_train_arr.npy")

# Load X_train and y_train arrays from disk
X_train = np.load("model-data/X_train_arr.npy")
y_train = np.load("model-data/y_train_arr.npy")

In [49]:
# Loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_train,
                                                              logits=conv_model)
optimizer = tf.train.AdamOptimizer().minimize()

SyntaxError: invalid syntax (<ipython-input-49-9693655e07e7>, line 4)